# 1.Load data

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
data = pd.read_csv('./OUTPUTAnnotations/dataset.csv', encoding= 'unicode_escape')
data.head()

,Sentence,Word,Tag
0,Sentence: 1,RECORD,0
1,Sentence: 2,OC,0
2,NaN,AM,0
3,NaN,gallstone,0
4,NaN,pancreatitis,0


## Rolling Window Preprocessing

In [3]:
data_fillna = data.fillna(method='ffill', axis=0)
data_group = data_fillna.groupby(['Sentence'],as_index=False
                                )['Word', 'Tag'].agg(lambda x: list(x))

#data_fillna
data_group.head()

C:\Users\jonat\AppData\Local\Temp/ipykernel_15748/2003111703.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(['Sentence'],as_index=False


,Sentence,Word,Tag
0,Sentence: 1,[RECORD],[0]
1,Sentence: 10,"[WILL, D/C, ORDER, BE, USED, AS, THE, D/C, SUM...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Sentence: 100,"[prandial, N/V/severe, upper, abdominal, pain....","[0, 1, 0, 1, 1, 0, 0, 0, 0]"
3,Sentence: 1000,"[normal, limits., Cardiac, catheterization, da...","[0, 0, 0, 0, 0, 0, 0, 0]"
4,Sentence: 10000,"[year, old, Black, female, with, significant, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [18]:
texts = data_group['Word'].tolist()  
labels = data_group['Tag'].tolist()

In [21]:
tokensAll = []
tagsAll = []
for tokenLine, tagLine in zip(texts, labels):
    for token, tag in zip(tokenLine, tagLine):
        tokensAll.append(token)
        tagsAll.append(tag)

In [24]:
indexSequence = 0
tokens607 = []
tags607 = []
count = 0
tokens = []
tags = []
firstTime = 0
appended = 0
while indexSequence < len(tokensAll):
    if count != 512:
        tokens.append(tokensAll[indexSequence])
        tags.append(tagsAll[indexSequence])
        indexSequence = indexSequence + 1
        count = count + 1
    elif count == 512 and firstTime == 0:
        tokens607 = np.array(tokens)
        tags607 = np.array(tags)
        firstTime = 1
        tokens = []
        tags = []
        indexSequence = indexSequence-112
        count = 0 
        appended = 1
        print(appended)
    elif count == 512 and firstTime == 1:
        tokens607 = np.vstack((tokens607, np.array(tokens)))
        tags607 = np.vstack((tags607, np.array(tags)))
        tokens = []
        tags = []
        indexSequence = indexSequence-112
        count = 0 
        appended = appended + 1
        print(appended)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


In [25]:
print('Shape of data tensor:', tokens607.shape)
print('Shape of label tensor:', tags607.shape)

Shape of data tensor: (2374, 512)
Shape of label tensor: (2374, 512)


In [28]:
dataBert = pd.DataFrame(columns = ['Stride', 'Word', 'Tag'])
dataBert

,Stride,Word,Tag


In [29]:
stride = 1
count = 0
while count < len(tokens607):
    dataBert.loc[len(dataBert.index)] = ['Stride: ' + str(stride), tokens607[count], tags607[count]] 
    count = count + 1
    stride = stride + 1

In [30]:
dataBert

,Stride,Word,Tag
0,Stride: 1,"[RECORD, WILL, D/C, ORDER, BE, USED, AS, THE, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."
1,Stride: 2,"[well, expanded., He, was, found, to, be, hypo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Stride: 3,"[EMSSten, Tel, Dictated, By:, QUARRY, FERNANDO...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Stride: 4,"[lymph, node., She, should, have, repeat, CT, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Stride: 5,"[room, air., The, patient, appeared, in, no, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
2369,Stride: 2370,"[weeks, of, increase, in, her, postnasal, drip...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, ..."
2370,Stride: 2371,"[The, urinalysis, revealed, too, numerous, to,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2371,Stride: 2372,"[all., This, revealed, that, she, also, has, c...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2372,Stride: 2373,"[signs:, Temperature, degrees, blood, pressure...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## BERT Tokenizer

In [84]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(dataBert, test_size=0.3, random_state=17)

In [85]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(data_train)
dataset_test = Dataset.from_pandas(data_test)

In [86]:
dataset_train[0]

{'Stride': 'Stride: 352',
 'Word': ['CV:',
  'tni',
  'slightly',
  'elevated',
  'but',
  'down',
  'to',
  'likely',
  'demand',
  'Tylenol',
  'no.',
  'one',
  'pill',
  'h.',
  'or',
  'two',
  'to',
  'three',
  'pills',
  'b.i.d.',
  'The',
  'patient',
  'has',
  'tried',
  'to',
  'limit',
  'narcotics',
  'to',
  'six',
  'tablets',
  'day.',
  'PAST',
  'MEDICAL',
  'HISTORY:',
  'Hypertension.',
  'Diabetes',
  'mellitus.',
  'Hypercholesteremia.',
  'Retinitis',
  'pigmentosa.',
  'Benign',
  'prostatic',
  'hypertrophy.',
  'Nephrolithiasis.',
  'Morbid',
  'obesity.',
  'ischemia',
  'in',
  'setting',
  'of',
  'stress.',
  'Cont',
  'ASA',
  'plavix',
  'BB',
  'NTG',
  'prn',
  'long',
  'Microhematuria.',
  'PAST',
  'SURGICAL',
  'HISTORY:',
  'Eye',
  'surgery.',
  'Knee',
  'surgery.',
  'Kidney',
  'stones.',
  'ADMISSION',
  'MEDICATIONS:',
  'Cardizem',
  'CD',
  'mg',
  'd.',
  'Prinivil',
  'mg',
  'b.i.d.',
  'Lipitor.',
  'Docusate.',
  'acting',
  'nitrate

In [87]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\jonat/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.15.0",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\jonat/.cache\huggingface\transformers\0e1bbfda7f6

In [88]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["Word"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"Tag"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:                            # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:              # Only label the first token of a given word.
                label_ids.append(label[word_idx])

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [89]:
tokenized_data_train = dataset_train.map(tokenize_and_align_labels, batched=True)
tokenized_data_test = dataset_test.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [90]:
tokenized_data_train[0]

{'Stride': 'Stride: 352',
 'Word': ['CV:',
  'tni',
  'slightly',
  'elevated',
  'but',
  'down',
  'to',
  'likely',
  'demand',
  'Tylenol',
  'no.',
  'one',
  'pill',
  'h.',
  'or',
  'two',
  'to',
  'three',
  'pills',
  'b.i.d.',
  'The',
  'patient',
  'has',
  'tried',
  'to',
  'limit',
  'narcotics',
  'to',
  'six',
  'tablets',
  'day.',
  'PAST',
  'MEDICAL',
  'HISTORY:',
  'Hypertension.',
  'Diabetes',
  'mellitus.',
  'Hypercholesteremia.',
  'Retinitis',
  'pigmentosa.',
  'Benign',
  'prostatic',
  'hypertrophy.',
  'Nephrolithiasis.',
  'Morbid',
  'obesity.',
  'ischemia',
  'in',
  'setting',
  'of',
  'stress.',
  'Cont',
  'ASA',
  'plavix',
  'BB',
  'NTG',
  'prn',
  'long',
  'Microhematuria.',
  'PAST',
  'SURGICAL',
  'HISTORY:',
  'Eye',
  'surgery.',
  'Knee',
  'surgery.',
  'Kidney',
  'stones.',
  'ADMISSION',
  'MEDICATIONS:',
  'Cardizem',
  'CD',
  'mg',
  'd.',
  'Prinivil',
  'mg',
  'b.i.d.',
  'Lipitor.',
  'Docusate.',
  'acting',
  'nitrate

In [91]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [92]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\jonat/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.15.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\jonat/.cache\huggingface\transfor

In [93]:
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[l for l in label if l != -100] for label in labels]
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [94]:
training_args = TrainingArguments(
    output_dir='./epochs',
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    save_steps= 1000,
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [95]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data_train,
    eval_dataset=tokenized_data_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

***** Running training *****
  Num examples = 1661
  Num Epochs = 10
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 2
  Total optimization steps = 60


ValueError: too many dimensions 'str'

In [27]:
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass

        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.9, 0.1]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [28]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data_train,
    eval_dataset=tokenized_data_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: Tag, Sentence, __index_level_0__, Word.
***** Running training *****
  Num examples = 92465
  Num Epochs = 2
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 2
  Total optimization steps = 722


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument weight in method wrapper_nll_loss_forward)